In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization
import keras
import pymorphy2
import deeppavlov
from keras import backend as K

In [ ]:
def normalize(word, morph=0): 
    if morph == 0:
        morph = pymorphy2.MorphAnalyzer()
    return morph.parse(word)[0].normal_form 

def json_to_russian_text(data):
    data['russian_text'] = 0
    morph = pymorphy2.MorphAnalyzer()
    for i in data.index:
        story = data['story_json'][i]
        story = story.lower()
        for symbol in 'qazxswedcvfrtgbnhyujmkiolp"{}[],.:;1234567890-/?=_#«»\\%&()–₽—$²':
            story = story.replace(symbol, ' ')
        story = story.split()
        story = ' '.join(story)
        data.loc[i, 'russian_text'] = story
    return data

In [ ]:
stories = json_to_russian_text(pd.read_csv("stories_description.csv"))

In [ ]:
from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder

In [ ]:
elmo = ELMoEmbedder("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz")

In [ ]:
text = stories.russian_text.apply(str.split).values.tolist()

In [ ]:
text_embeddings = np.array(elmo(text))

In [ ]:
data_text = pd.DataFrame(text_embeddings, columns=text_cols)

In [ ]:
data_text['story_id'] = stories['story_id']

In [ ]:
data_text.to_csv("text_feat_1024.csv", index=False)